# Step Forward Feature Selection(SFS)

On this notebook we identify possible best features for our model using the Step Forward Feature Selection (SFS)algorithm.

SFS is a wrapper method for feature selection and comes in two forms: <b/>Step Forward and Step Backward</b> feature selection.

Both these methods use the the same SFS object and can be specified by setting the forward parameter to True or False.

Our objective here is to:

    1. Reduce the feature space of our problem
    2. Find the best combination of features that predict our output

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
import matplotlib.pyplot as plt
from helpers import helpers
from sklearn.utils import resample
import warnings
#import statsmodels.api as sm

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [6]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

#dispaly all digits
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings('ignore')

In [5]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           9041        3309        1175         122        4555        5309
Swap:          2047           0        2047


# Get Data

In [7]:
data = pd.read_csv('derived_data/engineered.csv')

print(data.shape)

(452670, 14)


In [8]:
data.sample(10)

,national_inv,lead_time,in_transit_qty,forecast_3_month,sales_3_month,perf_6_month_avg,deck_risk_Yes,neg_inv_balance,lead_time_low,national_inv_low,in_transit_low,high_forecast,low_performance,went_on_backorder_Yes
349840,0.000000,2.000000,0.000000,3.000000,0.000000,0.570000,1.000000,0.000000,1,1,1,0,1,1
423078,0.000000,8.000000,0.000000,3.000000,2.000000,0.730700,0.000000,0.000000,0,1,1,0,1,1
21405,0.000000,2.000000,0.000000,13.000000,0.000000,0.850000,1.000000,0.000000,1,1,1,0,0,0
145473,30.000000,2.000000,0.000000,0.000000,4.000000,0.970000,0.000000,0.000000,1,0,1,0,0,0
393210,0.000000,2.000000,0.000000,10.000000,0.000000,0.085374,1.000000,0.000000,1,1,1,0,1,1
243592,7.000000,8.000000,0.000000,0.000000,4.000000,0.780000,0.000000,0.000000,0,1,1,0,0,0
437747,0.000000,4.000000,0.000000,6.000000,0.000000,0.830313,1.000000,0.000000,1,1,1,0,0,1
283181,360.000000,8.000000,0.000000,0.000000,114.000000,0.980000,0.000000,0.000000,0,0,1,0,0,0
162010,238.000000,8.000000,0.000000,0.000000,36.000000,0.700000,0.000000,0.000000,0,0,1,0,1,0
435212,50.947098,12.000000,0.000000,82.000000,114.000000,0.905239,0.000000,0.947098,0,0,1,1,0,1


In [10]:
#get discriptive stats
data.describe().T

,count,mean,std,min,25%,50%,75%,max
national_inv,452670.000000,53.612942,101.346334,0.000000,2.000000,8.000000,42.000000,360.000000
lead_time,452670.000000,6.835102,3.322310,2.000000,3.000000,8.000000,8.000000,12.000000
in_transit_qty,452670.000000,1.751648,4.600637,0.000000,0.000000,0.000000,0.000000,16.000000
forecast_3_month,452670.000000,16.712809,28.262880,0.000000,0.000000,0.000000,18.000000,82.000000
sales_3_month,452670.000000,20.700724,36.093728,0.000000,0.000000,3.000000,19.285180,114.000000
perf_6_month_avg,452670.000000,0.762177,0.241833,0.000000,0.680000,0.840000,0.950000,0.990000
deck_risk_Yes,452670.000000,0.208072,0.400302,0.000000,0.000000,0.000000,0.000000,1.000000
neg_inv_balance,452670.000000,0.026168,0.145039,0.000000,0.000000,0.000000,0.000000,1.000000
lead_time_low,452670.000000,0.331367,0.470705,0.000000,0.000000,0.000000,1.000000,1.000000
national_inv_low,452670.000000,0.491859,0.499934,0.000000,0.000000,0.000000,1.000000,1.000000


In [12]:
#get features and target
features = data.drop('went_on_backorder_Yes', axis=1)
target = data['went_on_backorder_Yes'].values

print(features.shape, target.shape)

(452670, 13) (452670,)


# Satandardize Data

In [14]:
#get list of column names
column_names = []
for col in features.columns:
    column_names.append(col)
    
column_names

['national_inv',
 'lead_time',
 'in_transit_qty',
 'forecast_3_month',
 'sales_3_month',
 'perf_6_month_avg',
 'deck_risk_Yes',
 'neg_inv_balance',
 'lead_time_low',
 'national_inv_low',
 'in_transit_low',
 'high_forecast',
 'low_performance']

In [15]:
features = features[column_names]

In [16]:
#get columns to standardize
standard_cols = column_names[:6]

standard_cols

['national_inv',
 'lead_time',
 'in_transit_qty',
 'forecast_3_month',
 'sales_3_month',
 'perf_6_month_avg']

In [20]:
#instantiate transformer object
ct = ColumnTransformer([
    ('standardize', StandardScaler(), standard_cols)
], remainder = 'passthrough')

In [21]:
#apply fit-transform
transformed_features = ct.fit_transform(features)

transformed_features

array([[-0.40073465,  1.55461231, -0.38074079, ...,  1.        ,
         0.        ,  1.        ],
       [-0.49940631,  0.35062941, -0.38074079, ...,  1.        ,
         0.        ,  1.        ],
       [-0.44020331,  0.65162514,  0.48870498, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.49842603, -1.45534494, -0.38074079, ...,  1.        ,
         1.        ,  1.        ],
       [-0.52252638,  0.35062941, -0.38074079, ...,  1.        ,
         1.        ,  1.        ],
       [-0.48953914, -1.45534494, -0.38074079, ...,  1.        ,
         0.        ,  0.        ]])

# Split Data

In [27]:
#train, test split
#set stratify by target to ensure proportions of target classes are maintained
X_train, X_test, y_train, y_test = train_test_split(transformed_features,target, test_size=0.2, \
                                                    random_state = 7, stratify=target)

print(X_train.shape, X_test.shape)

(362136, 13) (90534, 13)


In [29]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           9041        3614         815         142        4610        4983
Swap:          2047           0        2047


# Feature Selection

In [31]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           9041        3581         846         143        4613        5017
Swap:          2047           0        2047


In [34]:
#instantiate sfs object
#setting verbose = 2 returns scores as well
#use 'average_precision' scoring metric to give more weight to recall
sfs = SFS(LogisticRegression(solver='saga'),
         k_features = 7,
         forward = True,
         floating = False,
         verbose = 2,
         scoring = 'average_precision',
         cv = 4,
         n_jobs = -1
         ).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:   58.9s finished

[2020-08-20 11:21:47] Features: 1/7 -- score: 0.6192312728644027[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.1min finished

[2020-08-20 11:22:52] Features: 2/7 -- score: 0.7587954232389642[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:  1.1min finished

[2020-08-20 11:23:56] Features: 3/7 -- score: 0.784402915080376[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]:

From above, we see that highest score achieved was only 81.97% when all 7 features were selected.

Let's evaluate how our score (avergae_precision) changes when we select all 13 features.

**Took 7:18.84 mins to run

In [35]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           9041        3071        1335         145        4633        5523
Swap:          2047           0        2047


In [36]:
#instantiate sfs object
#setting verbose = 2 returns scores as well
#use 'average_precision' scoring metric to give more weight to recall
sfs = SFS(LogisticRegression(solver='saga'),
         k_features = 13,
         forward = True,
         floating = False,
         verbose = 2,
         scoring = 'average_precision',
         cv = 4,
         n_jobs = -1
         ).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:  1.1min finished

[2020-08-20 11:36:04] Features: 1/13 -- score: 0.6192312728644027[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.1min finished

[2020-08-20 11:37:07] Features: 2/13 -- score: 0.7587493274532047[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:  1.0min finished

[2020-08-20 11:38:10] Features: 3/13 -- score: 0.7843997436214799[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-

Highest score reached when at least 5 features were selected but not more than 11

The <b/>highest score of 82.08% was achieved when 8 features were selected.</b>

Note that with all 15 features included, our score droped to only 79.93% 

This demonstrates that it is not true always that the maximum number of selected features leads to best performance, the combination of selected features matters as well. 

There are actually a few subset of features that help a model perform really well.

This is why it is helpful to run this test so we can understand how impactful are the features we have in trying to solve the machine learning problem at hand and it is an important tool for a data scientist to have.

Thus, with <b/>Step Forward</b> feature selection, we can perform feature selection while improving accuracy as well.

Note that <b/>Step Backward</b> follows the same alrgorithm as Step Forward, but we just set the <b/>forward parameter</b> to <b/>False.</b>

Therefore, with the combination of features we have we can only achieve 82.07% with a <b/>logistic regression model</b> were <b/>solver is set as 'saga'</b> and other parameters are left as default.

Note, we use 'saga' because of the large size of our dataset.

To boost performance furthet, we might need to try either, 

    1. A different algorithm
    2. Engineer more relevant features
    
Let's get the identified <b/>best combination of selected features.</b>

*Note: Observe changes in traing time as well depending on number of selected features combination*

**took around 10 minutes to run

In [37]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           9041        3087        1323         139        4630        5514
Swap:          2047           0        2047


# Selected Features

In [38]:
#instantiate sfs object
#setting verbose = 2 returns scores as well
#use 'average_precision' scoring metric to give more weight to recall
sfs = SFS(LogisticRegression(solver='saga'),
         k_features = 8,
         forward = True,
         floating = False,
         verbose = 2,
         scoring = 'average_precision',
         cv = 4,
         n_jobs = -1
         ).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:  1.0min finished

[2020-08-20 11:58:48] Features: 1/8 -- score: 0.6192312728644027[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.0min finished

[2020-08-20 11:59:50] Features: 2/8 -- score: 0.7587590353333877[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:  1.0min finished

[2020-08-20 12:00:52] Features: 3/8 -- score: 0.7844025608732235[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]

In [39]:
#get selected feature names
sfs.k_feature_names_

('0', '2', '3', '4', '7', '9', '10', '11')

Note that feature names are not returned because we fed a numpy array to the SFS object.

If we convert the array back to a dataframe then feed the dataframe into SFS, sfs.K_feature_names will return the name of features. 

In our case now, this is ok as we can still get the selected column names by infering to the column indexes returned.

Note that an array is processed faster than a pandas dataframe and this is ok as we have a fairly large dataset to process

In [41]:
column_names

['national_inv',
 'lead_time',
 'in_transit_qty',
 'forecast_3_month',
 'sales_3_month',
 'perf_6_month_avg',
 'deck_risk_Yes',
 'neg_inv_balance',
 'lead_time_low',
 'national_inv_low',
 'in_transit_low',
 'high_forecast',
 'low_performance']

In [40]:
#let's get feature indexes
sfs.k_feature_idx_

(0, 2, 3, 4, 7, 9, 10, 11)

In [42]:
#let's get our score with those eight selected features
sfs.k_score_

0.8207781815080292

Here we see that the <b/>maximum average_precission</b> obtained by the above 8 features is 82.07%

Let's <b/>set up a pandas dataframe to view our results</b> more intuitively.

We do this  using a <b/>.from_dict()</b> method with an SFS dictionary object that has the information we need

In [43]:
#get performance metrics dataframe
pd.DataFrame.from_dict(sfs.get_metric_dict()).T

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(0,)","[0.6162188843140307, 0.6211337912200536, 0.619...",0.619231,"(0,)",0.002929,0.001827,0.001055
2,"(0, 4)","[0.7557481305430995, 0.7614912799984115, 0.761...",0.758759,"(0, 4)",0.004040,0.002521,0.001455
3,"(0, 4, 7)","[0.7833145481280287, 0.787239410653325, 0.7867...",0.784403,"(0, 4, 7)",0.004515,0.002816,0.001626
4,"(0, 2, 4, 7)","[0.7955999101641947, 0.798660934179299, 0.7997...",0.797152,"(0, 2, 4, 7)",0.003386,0.002112,0.001220
5,"(0, 2, 4, 7, 9)","[0.8061538792920192, 0.8088763536184177, 0.809...",0.807984,"(0, 2, 4, 7, 9)",0.002274,0.001419,0.000819
6,"(0, 2, 4, 7, 9, 11)","[0.8174840030552262, 0.8193620053595867, 0.820...",0.818744,"(0, 2, 4, 7, 9, 11)",0.002312,0.001442,0.000833
7,"(0, 2, 4, 7, 9, 10, 11)","[0.818191203944866, 0.8204250261417484, 0.8218...",0.819763,"(0, 2, 4, 7, 9, 10, 11)",0.002391,0.001492,0.000861
8,"(0, 2, 3, 4, 7, 9, 10, 11)","[0.8192776101853572, 0.8215132230056813, 0.822...",0.820778,"(0, 2, 3, 4, 7, 9, 10, 11)",0.001890,0.001179,0.000681


From above we see that for example, 

When 4 feature indexes (0,2,4, and 7) with thier names under the <b/>feature names</b> column, the model achieved an average score (average_precision) of 79.71%

Therefore, this method is very helpful for a data scientist during feature selection by providing intuition on how the features selected impact model performance

We can extract the feature names from our score metrics dataframe by assighning our dataframe to a variable and using .iloc() to slice our results as below. 

In [44]:
scores = pd.DataFrame.from_dict(sfs.get_metric_dict()).T

In [46]:
#get feature names of the combination of eight features
scores.iloc[7, 3]

('0', '2', '3', '4', '7', '9', '10', '11')

Therefore, the combination of these eight features can only explain 82% of the variablilty in back order

Thus improve performance further, we need to either:

    1. Think of additional features that can help explain most of the remaing 18% of unexplained variablity in back order, or
    2. Try to identify a better algorithm that can identify relationships in our data better (We'll try a CNN)

# Automate Feature Selection

Below is the code on how we can select best features automatically. 

Let's first convert pur scaled features back to a dataframe s our sfs object returns feature names instead of indeces.

Note that, we're not concerned with speed right now becaue we know that we only need eight features(k=8)

In [50]:
#get transformed features dataframe
transformed_feats_df = pd.DataFrame(data = transformed_features, columns = [column_names])

transformed_feats_df.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,sales_3_month,perf_6_month_avg,deck_risk_Yes,neg_inv_balance,lead_time_low,national_inv_low,in_transit_low,high_forecast,low_performance
0,-0.400735,1.554612,-0.380741,-0.591335,-0.573528,-1.166830,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
1,-0.499406,0.350629,-0.380741,-0.414424,-0.324176,-0.133056,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
2,-0.440203,0.651625,0.488705,-0.591335,-0.573528,0.363155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1.572699,0.651625,3.097042,-0.591335,2.584919,0.611261,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2.174596,0.350629,-0.380741,-0.591335,0.645522,0.776665,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [53]:
#split test, train sets
X_train, X_test, y_train, y_test = train_test_split(transformed_feats_df,target, test_size=0.2, \
                                                    random_state = 7, stratify=target)

print(X_train.shape, X_test.shape)

(362136, 13) (90534, 13)


In [54]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           9041        3170        1219         136        4651        5434
Swap:          2047           0        2047


In [55]:
#assign an interval to the parameter k_features
#we want SFS to evaluate performance from k = 1 feature to 8 features, i.e. use (1,8)
#SFS returns the best score for these selected feature combinations

sfs = SFS(LogisticRegression(solver='saga'),
         k_features = 8,
         forward = True,
         floating = False,
         verbose = 2,
         scoring = 'average_precision',
         cv = 4,
         n_jobs = -1
         ).fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:  1.1min finished

[2020-08-20 13:01:24] Features: 1/8 -- score: 0.6192312728644027[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.2min finished

[2020-08-20 13:02:34] Features: 2/8 -- score: 0.7587645603344891[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  11 | elapsed:  1.1min finished

[2020-08-20 13:03:39] Features: 3/8 -- score: 0.7844007178785533[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]

In [56]:
#get best k score
sfs.k_score_

0.8207768677149343

In [57]:
#get best k feature names
sfs.k_feature_names_

(('national_inv',),
 ('in_transit_qty',),
 ('forecast_3_month',),
 ('sales_3_month',),
 ('neg_inv_balance',),
 ('national_inv_low',),
 ('in_transit_low',),
 ('high_forecast',))

With best features identified, we'll take note of these results and apply them in our next modeling steps.